# Lade Daten per Twitter API
*Es werden Timeline Daten für jeden aktuellen Bundestags Politiker geladen, der auf Twitter aktiv ist. 
Die Daten werden anschließend in MongoDB abgelegt, wobei jeder Tweet einem Dokument entspricht. Die Namen der User wurden von den Fraktionsseiten gecrawled.*

### API Verbindung aufsetzen

In [1]:
# Tweepy statt twitter Paket
# https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/

import os
import tweepy as tw
import pandas as pd

# Die folgenden Zugangsaten können bei Twitter Developer abgerufen werden, sobald ein Account erstellt wurde
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token_key = os.getenv('access_token_key')
access_token_secret = os.getenv('access_token_secret')

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### API Verbindung und speichern in Mongo DB
In der Collection Twitter_mdp wird jeder Tweet mit allen Metainformationen als ein Dokument gespeichert. Es werden Tweets zu allen auf Twitter aktiven Politikern einer Fraktion gesammelt. Die entsprechenden Namen wurden von den Bundestagsfraktionsseiten gecrawled.

Anfang Juni 2020 wurde zur Qualitätsicherung des Datensatzes außerdem eine manuelle Kontrolle durchgeführt. Dabei fiel auf, dass einige Abgeordnete einen Twitter Account besitzen, dieser aber nicht auf der Fraktionsseite verlinkt wurde. Dies ist der Fall bei den folgenden Politikern:

"Philipp_Amthor", "reinhardbrandl", "gadechens", "EckhardGnodtke", "GuntherKrichba3", "AndreasLaemmel", "SaskiaLudwig17", "christophploss", "BerndSiebert", "VriesChristoph", "KEESdeVriesCDU", "TobiasZech" 

Die Tweets aus den Timelines dieser Politiker wurden entsprechend im Juni in den Datensatz aufgenommen.


In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

db = client['Twitter']
# All_Tweets_collection = db['twitter_mdp_ex_date_proj'] # Flache Collection mit weniger Feldern
All_Tweets_collection = db['Twitter_mdp_extend']         # Basis in der zunächst alle Tweets gesammelt werden

Über den Zeitraum Feb-Aug 2020 wurde der Datensatz zu verschiedenen Zeitpunkten aktualisiert. Dabei ergaben sich folgende Probleme:
* die Option "since" in $\texttt{.user_timeline}$ wird ignoriert wenn man gleichtzeitig "count" oder "page" verwendet
* beim laden in die modifizierte Collection Twitter_mp_datetime wurde ein duplicate error geworfen
* durch die Arbeit auf zwei Computern muss die Datenbank bei Aktualisierung synchronisiert werden

In [4]:
import json

with open('Namen_Union.json') as json_file:
    Union = json.load(json_file)
    
Union = Union[21:]
#print(Union)

for Abgeordneter in Union:
    i = 1
    user = Abgeordneter
    print(user)
    all_tweets = []
    while i <= 100 :
        tweets_of_page = api.user_timeline(screen_name = user, tweet_mode = "extended",  count = 200, page = i, include_rts = True)
        #print(len(tweets_of_page))
        all_tweets = all_tweets + tweets_of_page
        i=i+1
        if len(tweets_of_page) == 0:
            break    # no more tweets from this user
            
    for tweet in all_tweets:
        All_Tweets_collection.insert_one(tweet._json)
    print(len(all_tweets))
        


MichaelFrieser
245
fuchtel
1113
thomasgebhart
1415
groehe
3044
MGrosseBroemer
3211
GruebelMdb
154
FGuentzler
4
olavgutting
3191
ChristianHaase6
1
hahnflo
3224
juergenhardt
3194
MatthiasHauer
3223
MarkHauptmann
2747
MatthiasHeider
538
MechthildHeil
1873
ThomasHeilmann
454
FrankHeinrich
1185
MarcHenrichmann
444
ChristianHirte
3218
HHirte
3237
tj_tweets
3215
Ingmar_Jung
720
AnjaKarliczek
1698
KartesMdB
381
StefanKaufmann
3213
RonjaKemmer
922
RKiesewetter
3192
Kiessling_MdB
82
VolkmarKlein
286
axel_knoerig
74
JensKoeppen
3234
guenterkrings
43
RuedigerKruse
1828
MichaelKuffer
152
Dr_Roy_Kuehne
678
UlrichLange
2
DrLaunert
90
KLeikert
2468
DerLenzMdB
2641
AndreaLindholz
316
NikolasLoebel
539
JM_Luczak
3201
DaniLudwigMdB
1867
MaagKarin
181
YvonneMagwas
546
ManderlaGisela
714
AMattfeldt
127
meister_schafft
3219
JanMetzler
1022
hans_michelbach
184
MdBMonstadt
1
KarstenMoering
450
ElisabethMotsc1
98
mueller_sepp
3233
smuellermdb
1901
DrAndreasNick
3205
WilfriedOellers
697
Florian_Ossner
37
josefo

In [6]:
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/Union_twitter_clean.json') as json_file:
import json
with open('Namen_Union.json') as json_file:
    Union = json.load(json_file)
    
Union = Union[21]
print(Union)

HPFriedrichCSU


In [ ]:
user = api.user_timeline(screen_name = 'n_roettgen', since = "Tue Feb 04 06:30:24 +0000 2020", page = 1, count = 200, include_rts = True)
        
user_tw = [[tweet.created_at, tweet.in_reply_to_screen_name, tweet.text] for tweet in user]
df = pd.DataFrame(data=user_tw, columns=["created", "reply to", "tweet"])
df


#### Liste aller Unions Politiker ohne Daten in Twitter_mdp

Für manche Politiker findet man keine Daten in Twitter_mdp, wenn man mit dem Screen_name aus der Liste Union_twitter_clean sucht.

Grund: Die Usernamen in der Liste von der Fraktionsseite (_twitter_clean) weichen ab vom tatsächlichen Screen_name

-> Twitter API ist NICHT case sensitiv, MongoDB und Python aber schon!

=> Option mit RegEx zu suchen (_twitter_Namen) ODER einen Tweet laden und exakten user_screen_name speichern (siehe unten - _twitter_realNames)

In [7]:
# import json
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/Union_twitter_clean.json') as json_file:
#     Union = json.load(json_file)

# Union_Name = Union
# mdp_not_mongodb = []

# for name, n in zip(Union, range(0,len(Union))):
#     count = 0
#     #cursor = All_Tweets_collection.find({"user.screen_name": {'$regex':name,'$options':'i'}})
#     cursor = All_Tweets_collection.find({"user_screen_name": name})
#     for doc in cursor:
#         count=count+1
        
#     if count == 0:
#         print(count)
#         print(name)
#         #mdp_not_mongodb.append(name)
#         Union_Name[n] = All_Tweets_collection.find_one({"user_screen_name": {'$regex':name,'$options':'i'}})['user_screen_name']
   
        
# print(mdp_not_mongodb)
# with open("/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/Union_twitter_Names_test.json", "a") as write_file:
# #with open("mdp_not_mongodb_Union.json", "w") as write_file:
#      json.dump(Union_Name, write_file)



#### Erstellen der Liste mit den tatsächlichen screen_names 
Es wird ein Tweet geladen und anschließend dieser Screen_name in eine Liste Eingepflegt

In [6]:
# import json

# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/Union_twitter_clean.json') as json_file:
#     Union= json.load(json_file)
    
# #Union = Union[]
# names = []

# for Abgeordneter in Union:
#     user = Abgeordneter
#     print(user)
#     all_tweets = api.user_timeline(screen_name = user, count = 1, include_rts = True)
#     for tweet in all_tweets:
#         name = tweet.user.screen_name
#         names.append(name)
        
# print(names)

# with open ("Union_twitter_realNames", "w") as write_file:
#     json.dump(names, write_file)
    